In [1]:
%load_ext autoreload
%aimport data_creater
%aimport model
%autoreload 1

from data_creater import *
from model import *
import re, copy

Using TensorFlow backend.


In [3]:
stocks = companies()
symbols = stocks['Symbol'].values.tolist()

In [13]:
import gc 
# del df
# del stocks, Sequential
gc.collect()

199

In [93]:
stocks = companies()
symbols = stocks['Symbol'].values.tolist()
symbol = 'AAPL'
if os.path.isfile('./data/{0}/important_symbol.json'.format(symbol)):
    with open('./data/{0}/important_symbol.json'.format(symbol), 'r') as json_file:
        loaded_model_json = json_file.read()
        loaded_model_json = json.loads(loaded_model_json)
        symbols = list(loaded_model_json['score'].keys())

other_symbols = [symbol_tem for symbol_tem in symbols if symbol_tem != symbol]
df = pd.read_csv('./data/{0}/all_normalized.csv'.format(symbol), index_col=[0], parse_dates=[0])
symbol_columns = [symbol_tem for symbol_tem in list(df.columns.values) if bool(re.match('normal.*', symbol_tem))]

box = []
for symbol_tem in other_symbols:
    box_tem = []
    for col in list(df.columns.values):
        if bool(re.match(symbol_tem + '_normal_.*', col)):
            box_tem.append(col)
    box.append(box_tem)

all_combination = []
all_combination.append(symbol_columns)
for box_tem in box:
    all_combination.append(symbol_columns + box_tem)
    
all_combination[0]

['normal_amplitude',
 'normal_volume',
 'normal_close',
 'normal_close_diff_day1',
 'normal_close_diff_day2',
 'normal_close_diff_day3',
 'normal_returns',
 'normal_mfi',
 'normal_weekday']

In [6]:
ticker = 'AAPL'
window_size = 3
seq_obj = MultiSequence(ticker,window_size,1,'g000',all_combination[0])


# sys.exit()
X_train,y_train,X_test,y_test = split_data(seq_obj)
print(seq_obj.X[2])
print(seq_obj.y[2])





[[-0.03352107 -0.17754371 -0.3861563   0.70570341  0.54774041  0.66382023
   0.64942526 -0.68993871  3.        ]
 [-0.26815705 -0.2792827  -0.40817555  0.09049326  0.47626233  0.44672878
   0.12399471 -0.64910956  4.        ]
 [-0.60335345 -0.33233866 -0.42354147  0.12243274 -0.06654407  0.41229764
   0.15052712 -0.64951155  5.        ]]
[1]


In [7]:
window_sizes = [3,4]
dropouts =  [0.25]
learn_rates = [0.001]
epochs = [5,10,30]
batch_size = 50

In [11]:
result = model_selector('AAPL', window_sizes, learn_rates, dropouts, epochs, batch_size,target_length=1,target_theme='g001',verbose=1,column=all_combination[1])

print("\nResults : ")
print("-"*60)
print(result[0])

print(result[1])
ModelLoader.save(result[1]['ticker'],result[0],result[1],force_overwrite=False)


*** Best Model Selection for AAPL ***

Window size: 3
------------------------------------------------------------
1    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 5 Training loss: 4.2359 Training acc: 0.6632 Testing loss: 4.7219 Testing acc: 0.3469 Score : 0.4556
2    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 10 Training loss: 0.6138 Training acc: 0.7254 Testing loss: 0.6264 Testing acc: 0.6939 Score : 0.7093
3    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 30 Training loss: 0.6637 Training acc: 0.7513 Testing loss: 0.9840 Testing acc: 0.6939 Score : 0.7214

Window size: 4
------------------------------------------------------------
4    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 5 Training loss: 0.6055 Training acc: 0.6771 Testing loss: 0.9504 Testing acc: 0.6939 Score : 0.6854
5    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 10 Training loss: 0.6894 Training acc: 0.7083 Testing loss: 2.1238 Testing acc: 0.6939 Score : 0.7010
6    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 30 Training loss: 

In [72]:
import os, re
symbol = 'AAPL'
path = './model_step/{0}'.format(symbol)
files = []
for r, d, f in os.walk(path):
    for file in f:
        if 'props' in file:
            files.append(os.path.join(r, file))

df = pd.DataFrame(columns=['other_symbol', 'score'])
for i in range(len(files)):
    try:
        with open(files[i], 'r') as json_file:
            loaded_model_json = json_file.read()
            loaded_model_json = json.loads(loaded_model_json)

        other_symbol_name = '.'.join(list((set([col.split('_')[0] for col in loaded_model_json['colname'] if not re.match('normal.*', col)]))))
        if other_symbol_name == '':
            other_symbol_name = symbol
        score = loaded_model_json['score']
        df.loc[i] = [other_symbol_name, score]
        
        if score > 0.7:
            print(files[i])
        
    except:
        continue
df = df.sort_values(by = 'score', ascending = False).head(30)
df.index = df['other_symbol']
df = df.drop(columns = ['other_symbol'])
df_json = df.to_dict()
print(df_json)
with open('./data/{0}/important_symbol.json'.format(symbol), 'w', encoding='utf-8') as f:
    json.dump(df_json, f, ensure_ascii=False, indent=4)

{'score': {'VIAB': 0.6628, 'FRC': 0.6616, 'PPL': 0.653, 'LOW': 0.6452, 'CAG': 0.6438, 'MKTX': 0.6429, 'ALGN': 0.6427, 'DLR': 0.6422, 'NRG': 0.6407, 'DFS': 0.6398, 'ABT': 0.6366, 'MRO': 0.6363, 'ROL': 0.6362, 'DISCK': 0.6352, 'MDLZ': 0.6342, 'CME': 0.6327, 'AVY': 0.6327, 'AMD': 0.6325, 'DXC': 0.6323, 'HRB': 0.6319, 'MOS': 0.6317, 'FANG': 0.6314, 'CERN': 0.6311, 'INCY': 0.6308, 'UPS': 0.6288, 'PNR': 0.6276, 'AXP': 0.6267, 'TFX': 0.6267, 'HSIC': 0.6262, 'FAST': 0.6252}}
